In [11]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [16]:
#download stopwords and wordnet
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ganesh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ganesh\AppData\Roaming\nltk_data...


True

In [7]:
#Take out sample data from whole data to perform actions
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
90,Brought Philip Larkin to life in a way that is...,positive
485,"I don't expect a lot from ghost stories, but I...",negative
644,I've watched a lot of television in my 51 year...,positive
659,"While thinking of ""The Great Escape"" I allowed...",positive
699,WARNING:I advise anyone who has not seen the f...,negative


In [14]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [17]:
df = normalize_text(df)
df.head()

,review,sentiment
90,brought philip larkin life way worthy greatest...,positive
485,expect lot ghost story expect story make bit s...,negative
644,watched lot television year never much fun wee...,positive
659,thinking great escape allowed mind wander back...,positive
699,warning advise anyone seen film yet read comme...,negative


In [18]:
df['sentiment'].value_counts()

sentiment
negative    254
positive    246
Name: count, dtype: int64

In [19]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [20]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
90,brought philip larkin life way worthy greatest...,1
485,expect lot ghost story expect story make bit s...,0
644,watched lot television year never much fun wee...,1
659,thinking great escape allowed mind wander back...,1
699,warning advise anyone seen film yet read comme...,0


In [21]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [22]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [23]:
X.shape, y.shape

((500, 100), (500,))

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [26]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/ganeshsp7/Capstone-MLOps-Project.mlflow')
dagshub.init(repo_owner='ganeshsp7', repo_name='Capstone-MLOps-Project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Initialized MLflow to track repo "ganeshsp7/Capstone-MLOps-Project"

Repository ganeshsp7/Capstone-MLOps-Project initialized!

<Experiment: artifact_location='mlflow-artifacts:/f26093aae8c04f5786b1617b07fd47e1', creation_time=1752392356921, experiment_id='0', last_update_time=1752392356921, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [ ]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, artifact_path="model")


        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-07-13 13:18:24,641 - INFO - Starting MLflow run...
2025-07-13 13:18:25,126 - INFO - Logging preprocessing parameters...
2025-07-13 13:18:26,314 - INFO - Initializing Logistic Regression model...
2025-07-13 13:18:26,314 - INFO - Fitting the model...
2025-07-13 13:18:26,355 - INFO - Model training complete.
2025-07-13 13:18:26,358 - INFO - Logging model parameters...
2025-07-13 13:18:26,697 - INFO - Making predictions...
2025-07-13 13:18:26,700 - INFO - Calculating evaluation metrics...
2025-07-13 13:18:26,713 - INFO - Logging evaluation metrics...
2025-07-13 13:18:31,537 - INFO - Saving and logging the model...
2025-07-13 13:18:33,073 - ERROR - An error occurred: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}
Traceback (most recent call last):
  File "C:\Users\Ganesh\AppData\Local\Temp\ipykernel_10540\3657176524.py", line 48, in <module>
    mlflow.sklearn.log_model(model, name="model")
  File "c:\Users\Ganesh\anaconda3\envs\Mlops1\l

🏃 View run thoughtful-flea-777 at: https://dagshub.com/ganeshsp7/Capstone-MLOps-Project.mlflow/#/experiments/0/runs/e41be69288c74e3aa183ff0a29c98b2b
🧪 View experiment at: https://dagshub.com/ganeshsp7/Capstone-MLOps-Project.mlflow/#/experiments/0
